In [1]:
#Import necessary modules
import numpy as np
from scipy.sparse import diags, csr_matrix, kron
from scipy.linalg import solve_sylvester, schur, expm
from scipy.sparse.linalg import spsolve, cg
from scipy.integrate import quad
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import time
import random
import sys

def plot_sol(X,Y,U):
    plt.figure(0)
#     ax = plt.axes(projection='3d')
    xline = np.reshape(X, -1)
    yline = np.reshape(Y, -1)
    zline = np.reshape(U, -1)
#     plt.contourf(X, Y, U)
    plt.imshow(U, extent=[0,1,0,1], origin='lower')
    plt.colorbar()
    plt.axis(aspect='image')
    plt.xlabel('x')
    plt.ylabel('y')
#     ax.set_xlabel('x')
#     ax.set_ylabel('y')
#     ax.set_zlabel('u')
#     ax.plot3D(xline, yline, zline)

def compute_err(U, U_exact):
    n = len(U)
    h = 1/(n+1)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

def kron_prod_iterative(T, F, tol):
    n = len(T)
    start_time = time.time()
    F = np.reshape(F,-1)
    A = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n)) #A = I kron T + T^t kron I
    kron_time = time.time() - start_time
    U, info = cg(A, F, tol=tol) #Solve using scipy sparse conjugate grad
    end_time = time.time()
    solve_time = end_time - kron_time - start_time
    total_time = end_time - start_time
    timings = [kron_time, solve_time, total_time]
    return U, timings

def monteCarlo(n,M):
    
    #Define parameters
    h = 1/(n+1) #step size

    #Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
    x = np.linspace(h, 1-h, n)
    y = np.linspace(h, 1-h, n)

    #Create internal mesh (excludes boundaries)
    X, Y = np.meshgrid(x, y, indexing='ij')

    #Define tridiagonal matrix T
    diagonals = [[-2],[1],[1]]

    #Compute exact solution for comparison
    U_exp = np.sin(np.pi*X) * np.sin(np.pi*Y)
#     U_std = (1/(np.sqrt(3))) * np.sin(3*np.pi*X) * np.sin(5 * np.pi * Y)

    #Define mean, variance and solutions as zero matrices
    mean_sol = np.zeros([n,n])
#     std_sol = np.zeros([n,n])
#     sols = np.zeros([M,n,n])
        
    for i in range(0, M): 
        eps = random.uniform(-1, 1) #Generate random eps in range(r1,r2)
        T = np.multiply((-eps)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = np.reshape(kron_prod_iterative(T, F,1e-9)[0], (n,n))
#         sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
#     plot_sol(X,Y,mean_sol)
    
#     #Calculate variance
#     for i in range(0, M):
#         std_sol += ((sols[i] - mean_sol)**2)
#     std_sol = std_sol/(M-1)
    
    error = compute_err(U_exp, mean_sol)[0]
#     error_std = compute_err(U_std, std_sol)
    print(error)
    
n = int(sys.argv[1])
M = int(sys.argv[2])

monteCarlo(n,M)








ValueError: invalid literal for int() with base 10: '-f'

## Monte Carlo - Single Parameter

In [8]:
def monteCarlo(n,M):
    
    #Define parameters
    h = 1/(n+1) #step size

    #Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
    x = np.linspace(h, 1-h, n)
    y = np.linspace(h, 1-h, n)

    #Create internal mesh (excludes boundaries)
    X, Y = np.meshgrid(x, y, indexing='ij')

    #Define tridiagonal matrix T
    diagonals = [[-2],[1],[1]]

    #Compute exact solution for comparison
    U_exp = np.sin(np.pi*X) * np.sin(np.pi*Y)
    U_std = (1/(np.sqrt(3))) * np.sin(3*np.pi*X) * np.sin(5 * np.pi * Y)

    #Define mean, variance and solutions as zero matrices
    mean_sol = np.zeros([n,n])
    std_sol = np.zeros([n,n])
    sols = np.zeros([M,n,n])
    
    random.seed(0)
        
    for i in range(0, M): 
        eps = random.uniform(-1, 1) #Generate random eps in range(r1,r2)
        T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())
        F = (2*(np.pi**2)*np.sin(np.pi*X)*np.sin(np.pi*Y))+(34*(np.pi**2)*(eps)*np.sin(3*np.pi*X)*np.sin(5*np.pi*Y))
        U = np.reshape(kron_prod_iterative(T, F,1e-9)[0], (n,n))
        sols[i] = U
        mean_sol += U
        
    mean_sol = mean_sol/M #Divide by number of samples to get the mean solution
#     plot_sol(X,Y,mean_sol)
    
    #Calculate variance
    for i in range(0, M):
        std_sol += ((sols[i] - mean_sol)**2)
    std_sol = std_sol/(M-1)
    std_sol = np.sqrt(std_sol)

    
    error_mean = compute_err(U_exp, mean_sol)[0]
#     error_std = compute_err(U_std, std_sol)[0]
    print(error_mean)

    

monteCarlo(10,100)
monteCarlo(100,1000)

# monteCarlo(10,25600)






0.19456999763055488
0.007320953282953457
